In [145]:
import pandas as pd
import datetime

In [383]:
#data must be in this form for the functions to work properly
sample_input_data = {
    "Date": {"example": "m/d/y", "type": "str"},
    "Tickers": {"example": "VTI", "type": "float"}
}

In [358]:
#CONSTANTS
CALC_YEARLY_RETURNS_OF = [20, 10, 5, 3, 1]

#**order of each portfolio breakdown must match order of data in csv right now
VG1_PORTFOLIO = {"VTI": 32, "VXUS": 22, "VMLUX": 14, "VWIUX": 18, "VWLUX": 14}
VG2_PORTFOLIO = {"VTSAX": 13.65, "VWUAX": 2.05, "VWNAX": 2.05, "VEXRX": .88, "VSEQX": .88, "VWILX": 1.95, "VTRIX": 1.95, "VTIAX": 9.10, "VUSFX": 50.00, "VMLUX": 5.25, "VWIUX":  7.00, "VWLUX": 5.25}

#setting a date used for analysis - only using the month to pick for calculating returns.  it calculates based off the first trading day of month selecte
date = datetime.datetime(2021, 9, 1)

In [565]:
#FUNCTIONS
def plan_returns(portfolio_analysis_summary_df, portfolio, plan_name):
    #drop the last row (portfolio weights)
    portfolio_analysis_summary_df = portfolio_analysis_summary_df.iloc[:-1, :]
    plan_returns = []

    for ticker in portfolio_analysis_summary_df.columns:

        ticker_returns = portfolio_analysis_summary_df[ticker] #a series
        ticker_portfolio_weight = portfolio[ticker] / 100 #a float
        # print(ticker, ticker_returns, ticker_portfolio_weight)
        plan_returns.append(ticker_returns * ticker_portfolio_weight)

    plan_totals = 0
    for sumprod in plan_returns:
        plan_totals += sumprod

    #rename series name
    plan_totals.name = plan_name
    return plan_totals

def portfolio_analysis(hist_data, portfolio, calc_yearly_returns_of, input_date_obj, filter_on="y", date_col_name="Date"):
    #convert date data to datetime obj
    hist_data[date_col_name] = pd.to_datetime(hist_data[date_col_name])
    #set date as index
    hist_data = hist_data.set_index(date_col_name, drop = False)
    hist_data = hist_data.sort_index()

    hist_data = _get_valid_dates(hist_data, input_date_obj)

    portfolio_returns = {}
    summary_key_name = "returns_summary"
    details_key_name = "returns_details"

    portfolio_returns[summary_key_name] = {}
    portfolio_returns[details_key_name] = {}

    for ticker in portfolio:

        if ticker == "Date":
            continue

        portfolio_returns[summary_key_name][ticker] = {}
        portfolio_returns[details_key_name][ticker] = {}

        for interval in calc_yearly_returns_of:

            #first filter on the filter option range
            start_year = input_date_obj.year - interval
            filtered_hist_data = hist_data.loc[hist_data[date_col_name].dt.year >= start_year] 
             
            portfolio_returns[summary_key_name][ticker]["average {} yr returns (%)".format(interval)] = round(filtered_hist_data[ticker].pct_change().mean() * 100, 2)
            portfolio_returns[details_key_name][ticker]["returns over {} yrs".format(interval)] = filtered_hist_data[ticker]

        portfolio_returns[summary_key_name][ticker]["portfolio_weight"] = portfolio[ticker]

    return portfolio_returns

    
def _get_valid_dates(hist_data, input_datetime_obj, filter_on="y", date_col_name="Date"):
    if filter_on == "y":
        #only filter on month passed from input datetime obj
        hist_data = hist_data.loc[hist_data[date_col_name].dt.month == input_datetime_obj.month]

        #loop through all data within the given month and filter on only the first trading date/month/yr
        #this is needed to make sure I don't pick days to filter on that are non-trading days
        #{year: index in hist_data}
        idx_of_first_month_trade = {}
        for index, row in hist_data.iterrows():
            if row["Date"].year not in idx_of_first_month_trade:
                idx_of_first_month_trade[row["Date"].year] = index

        #return df filtered on all the indexes of the data we want to use 
        return hist_data[hist_data.index.isin(idx_of_first_month_trade.values())]t

In [573]:
vanguard1 = pd.read_csv("price_data/vanguard1.csv")

vanguard1_analysis = portfolio_analysis(vanguard1, VG1_PORTFOLIO, CALC_YEARLY_RETURNS_OF, date)
vanguard1_analysis_summary = pd.DataFrame(vanguard1_analysis["returns_summary"])
vanguard1_analysis_summary

vanguard1_returns = plan_returns(vanguard1_analysis_summary, VG1_PORTFOLIO, "vanguard1")
vanguard1_analysis_summary

,VTI,VXUS,VMLUX,VWIUX,VWLUX
average 20 yr returns (%),10.31,4.42,0.08,0.45,0.42
average 10 yr returns (%),14.62,4.42,0.08,0.79,1.11
average 5 yr returns (%),16.37,7.70,0.28,0.44,0.32
average 3 yr returns (%),16.73,8.17,1.19,2.28,2.52
average 1 yr returns (%),30.60,23.55,0.09,0.75,1.32
portfolio_weight,32.00,22.00,14.00,18.00,14.00


In [575]:
vanguard2 = pd.read_csv("price_data/vanguard2.csv")
vanguard2_analysis = portfolio_analysis(vanguard2, VG2_PORTFOLIO, CALC_YEARLY_RETURNS_OF, date)
vanguard2_analysis_summary = pd.DataFrame(vanguard2_analysis["returns_summary"])

vanguard2_returns = plan_returns(vanguard2_analysis_summary, VG2_PORTFOLIO, "vanguard2")
vanguard2_analysis_summary

,VTSAX,VWUAX,VWNAX,VEXRX,VSEQX,VWILX,VTRIX,VTIAX,VUSFX,VMLUX,VWIUX,VWLUX
average 20 yr returns (%),10.31,11.22,5.23,8.03,8.42,9.70,5.46,4.43,0.13,0.08,0.45,0.42
average 10 yr returns (%),14.63,16.83,7.58,9.32,11.27,13.32,5.08,4.43,0.13,0.08,0.79,1.11
average 5 yr returns (%),16.36,21.98,6.59,12.71,10.52,21.55,7.30,7.68,0.13,0.28,0.44,0.32
average 3 yr returns (%),16.71,23.66,7.64,10.86,9.71,24.22,6.43,8.10,0.35,1.19,2.28,2.52
average 1 yr returns (%),30.51,21.07,30.18,33.15,45.32,27.34,29.83,22.14,-0.05,0.09,0.75,1.32
portfolio_weight,13.65,2.05,2.05,0.88,0.88,1.95,1.95,9.10,50.00,5.25,7.00,5.25


In [576]:
pd.DataFrame([vanguard1_returns, vanguard2_returns]).T

,vanguard1,vanguard2
average 20 yr returns (%),4.4226,2.710800
average 10 yr returns (%),5.9596,3.623297
average 5 yr returns (%),7.0956,4.412004
average 3 yr returns (%),8.0808,4.967731
average 1 yr returns (%),15.3054,9.136856


In [522]:
# checking with data pulled right from the NASDAQ
nasdaq = pd.read_csv("price_data/NASDAQ_VWUAX.csv")
nasdaq = nasdaq.sort_values("Date")
nasdaq = nasdaq.reset_index(drop = True)

#get only relevant data
nasdaq = nasdaq[["Date", "Close/Last"]]
#rename columns so they are consistent
nasdaq = nasdaq.rename(columns={"Close/Last": "VWUAX"})
#sort data so oldest is on top, if it is sorted differently the %change calculation change is differen
nasdaq = nasdaq.sort_index()

nasdaq_vwuax = portfolio_analysis(nasdaq, {"VWUAX": 11}, CALC_YEARLY_RETURNS_OF, date)
pd.DataFrame(nasdaq_vwuax["returns_summary"])

,VWUAX
average 1 yr returns (%),21.07
average 10 yr returns (%),17.18
average 20 yr returns (%),17.18
average 3 yr returns (%),23.66
average 5 yr returns (%),21.98
portfolio_weight,11.00


In [15]:
#adding rows to df
import pandas as pd

from numpy.random import randint
  
dict = {'TICK':['1 year', 'Tim', 'Rob', 'Georgia'],
        'cgr':[87, 91, 97, 95],
        'dir':[83, 99, 84, 76]
       }
  
df = pd.DataFrame(dict)
  
display(df)
  
df.loc[len(df.index)] = ['20 years', 1, .05] 
  
df

,TICK,cgr,dir
0,1 year,87,83
1,Tim,91,99
2,Rob,97,84
3,Georgia,95,76


,TICK,cgr,dir
0,1 year,87,83.00
1,Tim,91,99.00
2,Rob,97,84.00
3,Georgia,95,76.00
4,20 years,1,0.05


In [19]:
d = pd.DataFrame({"CG": [], "DI": [], "TOT": []})
d.loc[len(d.index)] = ['20 years', 1, .05] 
d


,CG,DI,TOT
0,20 years,1.0,0.05


In [21]:
from datetime import datetime

In [23]:
datetime(datetime.today().year, 1, 1)

datetime.datetime(2022, 1, 1, 0, 0)